INSTALLING GENAI

In [15]:
pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [16]:
import google.generativeai as genai

In [17]:
GOOGLE_API_KEY='your api key here'
genai.configure(api_key=GOOGLE_API_KEY)


In [18]:
# for m in genai.list_models():
#   if 'generateContent' in m.supported_generation_methods:
#     print(m.name)

In [19]:

# Model Configuration
MODEL_CONFIG = {
  "temperature": 0.2,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

## Safety Settings of Model
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [20]:
model = genai.GenerativeModel(model_name = "gemini-1.0-pro-vision-latest",
                              generation_config = MODEL_CONFIG,
                              safety_settings = safety_settings)

In [21]:
from pathlib import Path

def image_format(image_path):
    img = Path(image_path)

    if not img.exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/jpeg", ## Mime type are PNG - image/png. JPEG - image/jpeg. WEBP - image/webp
            "data": img.read_bytes()
        }
    ]
    return image_parts

In [22]:
def gemini_output(image_path, system_prompt, user_prompt):

    image_info = image_format(image_path)
    input_prompt= [system_prompt, image_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text

In [23]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """

image_path = r"C:\Users\hp\Desktop\ocr project\invoice_demo.jpg"

user_prompt = "What is the total amount in the image?"

gemini_output(image_path, system_prompt, user_prompt)

' 327.00'

### EXTRACTING WHOLE DATA IN JSON FROM **INVOICE**

In [24]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
image_path = r"C:\Users\hp\Desktop\ocr project\invoice_demo.jpg"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image"

In [25]:
output = gemini_output(image_path, system_prompt, user_prompt)
from IPython.display import Markdown
Markdown(output)

 ```json
{
  "InvoiceNo": "J0352019-K",
  "InvoiceDate": "11/01/2019",
  "InvoiceTime": "09:44:00",
  "Cashier": "USER",
  "Ref": "",
  "Customer": {
    "Name": "TAN CHAY YEE",
    "Address": "NO.53 JALAN PUTRA 1, TAMAN SRI PUTRA, 81200 JOHOR BAHRU, JOHOR",
    "GSTNo": "002116837376"
  },
  "Items": [
    {
      "ItemCode": "1072",
      "Description": "REPAIR ENGINE POWER SPRAYER (1UNIT)",
      "Quantity": 1,
      "UnitPrice": 160.00,
      "SalesPrice": 160.00,
      "Amount": 160.00,
      "Tax": 0.00
    },
    {
      "ItemCode": "70549",
      "Description": "GIANT 606 OVERFLOW ASSY",
      "Quantity": 1,
      "UnitPrice": 17.00,
      "SalesPrice": 17.00,
      "Amount": 17.00,
      "Tax": 0.00
    },
    {
      "ItemCode": "1071",
      "Description": "ENGINE OIL",
      "Quantity": 1,
      "UnitPrice": 10.00,
      "SalesPrice": 10.00,
      "Amount": 10.00,
      "Tax": 0.00
    },
    {
      "ItemCode": "70637",
      "Description": "GREASE FOR TOOLS 40ML (AKODA)",
      "Quantity": 1,
      "UnitPrice": 6.00,
      "SalesPrice": 6.00,
      "Amount": 6.00,
      "Tax": 0.00
    },
    {
      "ItemCode": "1643",
      "Description": "STARTER TALI",
      "Quantity": 1,
      "UnitPrice": 8.00,
      "SalesPrice": 8.00,
      "Amount": 8.00,
      "Tax": 0.00
    },
    {
      "ItemCode": "70197",
      "Description": "EY20 STARTER HANDLE",
      "Quantity": 2,
      "UnitPrice": 18.00,
      "SalesPrice": 36.00,
      "Amount": 36.00,
      "Tax": 0.00
    },
    {
      "ItemCode": "HD40 1L COTIN",
      "Description": "HD40 1L COTIN",
      "Quantity": 1,
      "UnitPrice": 9.00,
      "SalesPrice": 9.00,
      "Amount": 9.00,
      "Tax": 0.00
    }
  ],
  "TotalSales": 327.00,
  "Discount": 0.00,
  "Rounding": 0.00,
  "Total": 327.00,
  "Cash": 327.00,
  "Change": 0.00
}
```